In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
def get_CIFAR10(root="./"):
    input_size = 32
    num_classes = 10
    normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    
    train_transform = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]
    )
    train_dataset = datasets.CIFAR10(
        root + "data/CIFAR10", train=True, transform=train_transform, download=True
    )

    test_transform = transforms.Compose(
        [
            transforms.ToTensor(),
            normalize,
        ]
    )
    test_dataset = datasets.CIFAR10(
        root + "data/CIFAR10", train=False, transform=test_transform, download=True
    )

    return input_size, num_classes, train_dataset, test_dataset


In [2]:
def get_SVHN():
    dataset = datasets.SVHN
    num_classes = 10
    input_size = 32
    transform_train = transforms.Compose([
            transforms.RandomCrop(32,padding=4),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    train_dataset = dataset(root='./data', split='train', transform=transform_train,
                                     target_transform=None, download=True)
    test_dataset = dataset(root='./data', split='test', transform=transform_test,
                                   target_transform=None, download=True)
    return input_size,num_classes,train_dataset,test_dataset

In [165]:
input_size, num_classes, train_dataset, test_dataset = get_CIFAR10()
# input_size, num_classes, train_dataset, test_dataset = get_SVHN()

kwargs = {"num_workers": 2, "pin_memory": True}

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, **kwargs
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128, shuffle=False, **kwargs
)

Files already downloaded and verified
Files already downloaded and verified


In [126]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.nn.init as init

from torch.autograd import Variable
import sys

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)

def cfg(depth):
    depth_lst = [18, 34, 50, 101, 152]
    assert (depth in depth_lst), "Error : Resnet depth should be either 18, 34, 50, 101, 152"
    cf_dict = {
        '18': (BasicBlock, [2,2,2,2]),
        '34': (BasicBlock, [3,4,6,3]),
        '50': (Bottleneck, [3,4,6,3]),
        '101':(Bottleneck, [3,4,23,3]),
        '152':(Bottleneck, [3,8,36,3]),
    }

    return cf_dict[str(depth)]

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=True)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, depth, num_classes):
        super(ResNet, self).__init__()
        self.in_planes = 16

        block, num_blocks = cfg(depth)

        self.conv1 = conv3x3(3,16)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


In [157]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init
import numpy as np
import pdb

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)

def cfg(depth):
    depth_lst = [11, 13, 16, 19]
    assert (depth in depth_lst), "Error : VGGnet depth should be either 11, 13, 16, 19"
    cf_dict = {
        '11': [
            64, 'mp',
            128, 'mp',
            256, 256, 'mp',
            512, 512, 'mp',
            512, 512, 'mp'],
        '13': [
            64, 64, 'mp',
            128, 128, 'mp',
            256, 256, 'mp',
            512, 512, 'mp',
            512, 512, 'mp'
            ],
        '16': [
            64, 64, 'mp',
            128, 128, 'mp',
            256, 256, 256, 'mp',
            512, 512, 512, 'mp',
            512, 512, 512, 'mp'
            ],
        '19': [
            64, 64, 'mp',
            128, 128, 'mp',
            256, 256, 256, 256, 'mp',
            512, 512, 512, 512, 'mp',
            512, 512, 512, 512, 'mp'
            ],
    }

    return cf_dict[str(depth)]

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

class VGG(nn.Module):
    def __init__(self, depth, num_classes, dropout_rate = 0.):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg(depth), dropout_rate)
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.features(x)

        #below to work with datasets that have resolution other than 32x32
        m = torch.nn.AdaptiveAvgPool2d((1))
        out = m(out)
        #  --end 

        #pdb.set_trace()
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out

    def _make_layers(self, cfg, dropout_rate):
        layers = []
        in_planes = 3

        for x in cfg:
            if x == 'mp':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                if dropout_rate > 0.:
                    layers += [conv3x3(in_planes, x), nn.BatchNorm2d(x), 
                        nn.ReLU(inplace=True), nn.Dropout(p=dropout_rate)]
                else:
                    layers += [conv3x3(in_planes, x), nn.BatchNorm2d(x), nn.ReLU(inplace=True)]
                in_planes = x

        # After cfg convolution
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)



In [176]:
path="D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 2"
model = VGG(16, 11)#ResNet(18, 11)#
result=[]
for i in range(10):
    model=torch.load(f"resnet18_svhn_model_{i}.pt") #{path}/
    model=model.cuda()
    zipped=[]
    for data,label in test_loader:
        torch.cuda.empty_cache()
        with torch.no_grad():
            pre_softmax=model(data.cuda())  #
            softmax = F.softmax(pre_softmax, dim=1)
        softmax=softmax[:,:-1].cpu()            #
        entropy=(softmax*torch.log(softmax)).sum(1)
        GI=1-torch.pow(softmax,2).sum(1)

        top=(softmax-softmax.max(1)[0].unsqueeze(1)).sum(1).pow(2)
        mu_2=(1-softmax.max(1)[0])/(10-1)
        down=torch.pow(torch.sort(softmax)[0][:,:-1]-mu_2.unsqueeze(1),2).sum(1)
        LDAM=2*top/((10-1)*down)

        sorted_softmax=torch.sort(softmax)[0]-softmax.max(1)[0].unsqueeze(1)
        J_P=softmax.max(1)[0]-sorted_softmax.var(1)-sorted_softmax.mean(1)

        confidence=1-torch.sort(softmax)[0][:,-2]


        sorted_softmax=torch.sort(softmax)[0][:,:]
        Relative_diff=1-sorted_softmax[:,-2]/sorted_softmax[:,-1]

        zipped.extend(list(zip(softmax[:,-1],GI,entropy,LDAM,J_P,confidence,Relative_diff,softmax.max(1)[1].eq(label))))


    name=['extra class ','GI\t\t','entropy\t','LDAM\t','J_P\t\t','confidence\t','Relativ_diff']
    print('\t\t\t\t\tcoverage\n\t\t\t     98\t\t  95\t       90')
    data1=[]
    data2=[]
    for k in range(7):
        temp1=[]
        temp2=[]
        descending=True
        if k<2:
            descending=False#True#
        reverse=False
        if descending:
            reverse=True
        zipped.sort(key = lambda x: x[k],reverse=reverse)
        sorted_correct = list(map(lambda x: int(x[7]), zipped))
        size = len(sorted_correct)
        print(f'accracy for {name[k]} ',end='')
        expected_coverage=[98.,95.,90.]
        for coverage in expected_coverage:
            covered_correct = sorted_correct[:round(size/100*coverage)]
            print('{} {:.3f} |'.format(sum(covered_correct), sum(covered_correct)/len(covered_correct)*100.), end='')
            temp1.extend([sum(covered_correct)])
            temp2.extend([sum(covered_correct)/len(covered_correct)*100.])
        print('\n')
        data1.append(temp1)
        data2.append(temp2)
    result.append([data1,data2])

AttributeError: 'collections.OrderedDict' object has no attribute 'cuda'

In [167]:
res=torch.tensor(result)
res[:,0,1,1].mean()

tensor(8967.7002)

In [168]:
mn=res.mean(0)
std=torch.pow(res.var(0),0.5)
mn[0].int(),std[0].int()
mn[1],std[1]

(tensor([[93.0061, 94.1463, 95.7489],
         [93.1031, 94.3968, 96.1722],
         [93.1194, 94.4053, 96.1844],
         [93.0143, 94.3611, 96.1356],
         [93.0469, 94.3895, 96.1722],
         [92.9388, 94.3463, 96.1200],
         [93.0000, 94.3600, 96.1222]]),
 tensor([[0.2847, 0.2953, 0.2540],
         [0.3066, 0.2620, 0.1996],
         [0.3157, 0.2538, 0.2064],
         [0.2891, 0.2608, 0.1675],
         [0.3098, 0.2577, 0.2008],
         [0.2893, 0.2803, 0.1775],
         [0.2878, 0.2593, 0.1798]]))

In [169]:
d=[]

In [170]:
d.extend(mn[0].int().numpy())
d.extend(std[0].int().numpy())
d.extend(mn[1].numpy())
d.extend(std[1].numpy())

In [2]:
import pandas as pd
df=pd.DataFrame(d)

NameError: name 'd' is not defined

In [227]:
pd.DataFrame(b).to_csv(f'Gambler_4_6_resnet18_svhn.csv',index=False)

In [3]:
# file=['gambler_resenet18_svhn.csv','gambler_resenet18_cifar.csv','gambler_vgg16_svhn.csv','gambler_vgg16_cifar.csv']
# df=pd.read_csv(file[2])
path=["D:/OneDrive - Deakin University/review paper/data for paper/DAC",
      "D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 2",
      "D:/OneDrive - Deakin University/review paper/data for paper/gambler/reward 2"]
pre=['DAC','NWLoss','gambler']
c=pd.DataFrame()
for k in range(3):
    df=pd.read_csv(f'{path[k]}/{pre[k]}_resnet18_svhn.csv')

    b=[]
    name=['extra class ','GI\t\t','entropy\t','LDAM\t','J_P\t\t','confidence\t','Relativ_diff']
    idx=['extra class','GI','Entropy','LDAM','$J_P$','confidence','Relative difference']
    print('\t\t\t\t\t\tcoverage\n\t\t\t\t98\t\t 95\t       90')
    for i in range(7):
        print(f'accracy for {name[i]} ',end='')
        a=[]
        for j in range(3):
            print(df.iloc[i+14][j],'+-',df.iloc[(i+21)][j],end=' ')
            a.append("{:.3f}".format(df.iloc[i+14][j])+'$\pm$'+"{:.3f}".format(df.iloc[i+21][j]))
        b.append(a)
        
        print('\n')
    b=pd.DataFrame(b,columns=[98,95,90],index=idx)
    print(b)
    c=pd.concat([c,b], axis=1)

						coverage
				98		 95	       90
accracy for extra class  96.85979 +- 0.094428845 97.62394 +- 0.0891682 98.37637 +- 0.05986293 

accracy for GI		 97.04205 +- 0.096675485 98.11807 +- 0.06983652 98.95557 +- 0.07290774 

accracy for entropy	 96.995804 +- 0.09280185 98.041245 +- 0.07734083 98.92611 +- 0.077462144 

accracy for LDAM	 97.04755 +- 0.0860356 98.127785 +- 0.084155485 98.96837 +- 0.0755818 

accracy for J_P		 97.07969 +- 0.073983334 98.12373 +- 0.06854469 98.95643 +- 0.07229988 

accracy for confidence	 96.93584 +- 0.07526586 98.08451 +- 0.08428515 98.95813 +- 0.07385793 

accracy for Relativ_diff 97.03383 +- 0.081305444 98.122116 +- 0.08212722 98.96154 +- 0.075661786 

                                   98                95                90
extra class          96.860$\pm$0.094  97.624$\pm$0.089  98.376$\pm$0.060
GI                   97.042$\pm$0.097  98.118$\pm$0.070  98.956$\pm$0.073
Entropy              96.996$\pm$0.093  98.041$\pm$0.077  98.926$\pm$0.077
LDAM          

In [11]:
# path=["D:/OneDrive - Deakin University/review paper/data for paper/DAC",
#       "D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 2",
#       "D:/OneDrive - Deakin University/review paper/data for paper/gambler/reward 2"]
path=["D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 0.5",
      "D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 1.0",
      "D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 1.5",
     "D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha 2"]
pre='NWLoss'#['DAC','NWLoss','gambler']
c=pd.DataFrame()
for k in range(3):
    df=pd.read_csv(f'{path[k]}/{pre}_resnet18_svhn.csv')

    b=[]
    name=['extra class ','GI\t\t','entropy\t','LDAM\t','J_P\t\t','confidence\t','Relativ_diff']
    idx=['extra class','GI','Entropy','LDAM','$J_P$','confidence','Relative difference']
    print('\t\t\t\t\t\tcoverage\n\t\t\t\t98\t\t 95\t       90')
    for i in range(7):
        print(f'accracy for {name[i]} ',end='')
        a=[]
        for j in range(3):
            print(df.iloc[i+14][j],'+-',df.iloc[(i+21)][j],end=' ')
            a.append("{:.3f}".format(df.iloc[i+14][j])+'$\pm$'+"{:.3f}".format(df.iloc[i+21][j]))
        b.append(a)
        
        print('\n')
    b=pd.DataFrame(b,columns=[98,95,90],index=idx)
    print(b)
    c=pd.concat([c,b], axis=1)

						coverage
				98		 95	       90
accracy for extra class  96.8786 +- 0.103661835 97.67206 +- 0.09794291 98.42076 +- 0.052698523 

accracy for GI		 97.01854 +- 0.10541085 98.09219 +- 0.09467545 98.92142 +- 0.08517459 

accracy for entropy	 96.97972 +- 0.10544451 98.00809 +- 0.0968076 98.90137 +- 0.08529877 

accracy for LDAM	 97.037346 +- 0.11295872 98.08208 +- 0.107155085 98.93423 +- 0.0825202 

accracy for J_P		 97.045586 +- 0.10724039 98.09624 +- 0.09754865 98.923134 +- 0.08581563 

accracy for confidence	 96.91584 +- 0.12336026 98.04933 +- 0.10828774 98.92825 +- 0.08427679 

accracy for Relativ_diff 97.02442 +- 0.11236334 98.07885 +- 0.10533298 98.93252 +- 0.08104442 

                                   98                95                90
extra class          96.879$\pm$0.104  97.672$\pm$0.098  98.421$\pm$0.053
GI                   97.019$\pm$0.105  98.092$\pm$0.095  98.921$\pm$0.085
Entropy              96.980$\pm$0.105  98.008$\pm$0.097  98.901$\pm$0.085
LDAM               

In [295]:
c.to_csv(f'NWLoss_1_15_resnet18_svhn.csv',index=True)

In [12]:
c
# "{:.2f}".format(df.iloc[14][0])+'$\pm$'+"{:.2f}".format(df.iloc[21][0])

,98,95,90,98,95,90,98,95,90
extra class,96.879$\pm$0.104,97.672$\pm$0.098,98.421$\pm$0.053,96.888$\pm$0.122,97.663$\pm$0.087,98.398$\pm$0.051,96.857$\pm$0.105,97.604$\pm$0.091,98.359$\pm$0.085
GI,97.019$\pm$0.105,98.092$\pm$0.095,98.921$\pm$0.085,97.068$\pm$0.105,98.107$\pm$0.084,98.930$\pm$0.058,97.043$\pm$0.088,98.097$\pm$0.087,98.912$\pm$0.042
Entropy,96.980$\pm$0.105,98.008$\pm$0.097,98.901$\pm$0.085,97.018$\pm$0.115,98.039$\pm$0.097,98.906$\pm$0.056,97.007$\pm$0.090,98.005$\pm$0.085,98.892$\pm$0.036
LDAM,97.037$\pm$0.113,98.082$\pm$0.107,98.934$\pm$0.083,97.047$\pm$0.102,98.105$\pm$0.091,98.936$\pm$0.060,97.042$\pm$0.090,98.097$\pm$0.086,98.924$\pm$0.050
$J_P$,97.046$\pm$0.107,98.096$\pm$0.098,98.923$\pm$0.086,97.070$\pm$0.098,98.110$\pm$0.089,98.930$\pm$0.057,97.062$\pm$0.097,98.105$\pm$0.081,98.911$\pm$0.043
confidence,96.916$\pm$0.123,98.049$\pm$0.108,98.928$\pm$0.084,96.931$\pm$0.126,98.067$\pm$0.092,98.935$\pm$0.060,96.933$\pm$0.103,98.057$\pm$0.093,98.918$\pm$0.057
Relative difference,97.024$\pm$0.112,98.079$\pm$0.105,98.933$\pm$0.081,97.035$\pm$0.106,98.099$\pm$0.096,98.936$\pm$0.060,97.028$\pm$0.093,98.090$\pm$0.086,98.919$\pm$0.054


In [ ]:
file=['gambler_resenet18_svhn.csv','gambler_resenet18_cifar.csv','gambler_vgg16_svhn.csv','gambler_vgg16_cifar.csv']